In [1]:
import numpy as np
import pandas as pd

import nltk
import ftfy
import re
import inflect
from nltk.corpus import stopwords
from num2words import num2words
from nltk.tokenize import word_tokenize

from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import (
    ConfusionMatrixDisplay, RocCurveDisplay,
    roc_auc_score, precision_score, recall_score, f1_score
)
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.pipeline import Pipeline

In [2]:
pd.set_option("display.max_colwidth", None)

In [3]:
nltk.download("punkt")
nltk.download("stopwords")
stop_words = stopwords.words("spanish")

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\leopo\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\leopo\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
stop_words

['de',
 'la',
 'que',
 'el',
 'en',
 'y',
 'a',
 'los',
 'del',
 'se',
 'las',
 'por',
 'un',
 'para',
 'con',
 'no',
 'una',
 'su',
 'al',
 'lo',
 'como',
 'más',
 'pero',
 'sus',
 'le',
 'ya',
 'o',
 'este',
 'sí',
 'porque',
 'esta',
 'entre',
 'cuando',
 'muy',
 'sin',
 'sobre',
 'también',
 'me',
 'hasta',
 'hay',
 'donde',
 'quien',
 'desde',
 'todo',
 'nos',
 'durante',
 'todos',
 'uno',
 'les',
 'ni',
 'contra',
 'otros',
 'ese',
 'eso',
 'ante',
 'ellos',
 'e',
 'esto',
 'mí',
 'antes',
 'algunos',
 'qué',
 'unos',
 'yo',
 'otro',
 'otras',
 'otra',
 'él',
 'tanto',
 'esa',
 'estos',
 'mucho',
 'quienes',
 'nada',
 'muchos',
 'cual',
 'poco',
 'ella',
 'estar',
 'estas',
 'algunas',
 'algo',
 'nosotros',
 'mi',
 'mis',
 'tú',
 'te',
 'ti',
 'tu',
 'tus',
 'ellas',
 'nosotras',
 'vosotros',
 'vosotras',
 'os',
 'mío',
 'mía',
 'míos',
 'mías',
 'tuyo',
 'tuya',
 'tuyos',
 'tuyas',
 'suyo',
 'suya',
 'suyos',
 'suyas',
 'nuestro',
 'nuestra',
 'nuestros',
 'nuestras',
 'vuestro'

### Carga y preparación de datos

In [5]:
data = pd.read_excel("data/cat_6716.xlsx")

In [6]:
data.columns

Index(['Textos_espanol', 'sdg'], dtype='object')

In [7]:
# mostrar la fila 7 del dataframe
data.iloc[5]

Textos_espanol    La descentralizaciÃ³n tambiÃ©n implica dar mayor protagonismo a los ComitÃ©s TÃ©cnicos de Aguas SubterrÃ¡neas (COTAS) en la gestiÃ³n de los acuÃferos, fortalecer la organizaciÃ³n y funcionamiento de los Consejos de Cuenca y sus Ã³rganos auxiliares, y consolidar las funciones de gobernanza y organizaciÃ³n regional de la CONAGUA. No existe un modelo Ãºnico para mejorar la gobernanza pÃºblica en el sector hÃdrico y superar los obstÃ¡culos institucionales para su implementaciÃ³n, pero es fundamental diagnosticar los principales retos de gobernanza que se experimentan entre los ministerios y organismos pÃºblicos, entre los niveles de gobierno y entre los actores subnacionales, y revisar los instrumentos de gobernanza existentes. Esto ayudará a definir claramente las funciones y responsabilidades de las autoridades públicas y a establecer mecanismos de resolución de conflictos.
sdg                                                                                              

Revisión de los datos. Revisamos si existe algun texto que no tenga clasificación y no sea útil para determinar los SDG de los textos al momento de entrenar el modelo.

In [8]:
data['sdg'].value_counts()

6     1000
7     1000
16    1000
Name: sdg, dtype: int64

Se puede observar que hay una distribución de 1/3 de los datos por cada una de las categorias a analizar y no hay valores nulos. Es un buen indicativo para realizar predicciones.

In [9]:
data.shape

(3000, 2)

In [10]:
data.sample(5)

,Textos_espanol,sdg
2653,"La asistencia para el desarrollo de China en África ha atraído diversas críticas de académicos, gobiernos occidentales y organizaciones internacionales de donantes. Las principales críticas se han dirigido a la falta de buenas condiciones de gobernanza en su asistencia para el desarrollo a los gobiernos africanos y sus tratos con países sancionados o aislados de la comunidad internacional, debido a la mala gobernanza, los abusos de los derechos humanos y/o la corrupción. Utilizando el caso de Zimbabue en el período actual, se evalúa el impacto de la asistencia para el desarrollo de China en sus perspectivas de desarrollo sostenible. Desde el año 2000, Zimbabue ha experimentado una crisis de gobernabilidad que condujo a un declive socioeconómico y político. Podría decirse que esta crisis se ha visto exacerbada por la variada asistencia de China al régimen de Robert Mugabe, así como por la protección de China del régimen en el Consejo de Seguridad de las Naciones Unidas. Se argumenta que sin fomentar la buena gobernanza y gestionar esta relación en beneficio de Zimbabue, Harare no verá...",16
2170,"La gama de actividades realizadas por las operaciones multilaterales de paz se ha incrementado significativamente en los últimos 20 años, un aspecto importante de esto es la detención de personas. Sin embargo, durante mucho tiempo no quedó claro qué normas y leyes se aplican a las detenciones en las operaciones de mantenimiento de la paz. Solo recientemente han surgido los principios del estado de derecho dentro de las Naciones Unidas (ONU) y entre los estados implementadores. El capítulo de Hirschmann demuestra cómo la dinámica de la rendición de cuentas contribuyó a un estado de derecho cada vez mayor para las detenciones. Muestra cómo el establecimiento de normas, el control y las sanciones por parte de organizaciones regionales, organizaciones no gubernamentales y tribunales internacionales contribuyeron a los elementos básicos del estado de derecho, a saber, el Boletín del Secretario General de las Naciones Unidas sobre la observancia del derecho internacional humanitario, disposiciones específicas para la operación en Kosovo y el Proceso de Copenhague sobre el tratamiento de los detenidos.",16
619,"Esto se destacó en el primer estudio de caso, en el que el marco institucional bien establecido para la gestión del agua en los Países Bajos fue altamente propicio para la introducción exitosa de cargos por efluentes. En contraste, las diferentes tradiciones de gestión en Dinamarca, Francia y Alemania complicaron el uso de instrumentos económicos similares en esos países (Andersen, 2001). La experiencia con las iniciativas de comercialización de agua sugiere que los regímenes de comercialización exitosos tienden a basarse en instituciones preexistentes y se integran en los regímenes regulatorios tradicionales. Esto puede implicar que los permisos negociables se combinen con otros instrumentos de política (impuestos, cargos, objetivos de calidad ambiental, etc.)",6
648,"El concepto de actio popularis está reconocido en la Constitución de 1976 y su alcance se ha ampliado gradualmente. Una ley de 1998 relativa a las ONG medioambientales establece claramente las condiciones en las que se puede ejercer este derecho. Por ejemplo, las ONG (tengan o no un interés directo) pueden presentar una queja o denuncia e iniciar las acciones legales necesarias para la prevención, corrección, suspensión o cesación de acciones y omisiones por parte de entidades públicas y privadas que representen o puedan representar , una causa de la degradación ambiental.",6
2447,"De acuerdo con las disposiciones establecidas en el artículo 19 de la Convención de Viena sobre el Derecho de los Tratados de 1969, un Estado que consiente en obligarse por un tratado no puede formular tales reservas al acuerdo, que sean incompatibles con su objeto y fin. Este llamado 'criterio de objeto y fin' ha desconcertado durante mucho tiempo a los actores del 

## Correción de palabras para eliminación de caracteres extraños.

In [11]:
data['Textos_espanol'] = data['Textos_espanol'].astype(str)

In [12]:
def fix_malformed_words(text):
    text = ftfy.fix_text(text)
    return text

data['Textos_espanol'] = data['Textos_espanol'].apply(fix_malformed_words)

In [13]:
data.head(5)

,Textos_espanol,sdg
0,"Es importante destacar que, en un año de sequía, se espera que disminuyan todos los aportes, pero que aumente la demanda de agua de los cultivos y de la vegetación, lo que provocará una reducción del caudal de los ríos que están conectados hidrológicamente con el acuífero, así como una menor disponibilidad de agua para otros ecosistemas dependientes de las aguas subterráneas. Estas externalidades espaciales pueden incluir el agotamiento de los cursos de agua y el descenso del nivel freático local. Por ejemplo, el bombeo continuo del pozo B provocará el agotamiento de los cursos de agua y la reducción de su caudal.",6
1,"Hay una gran cantidad de literatura sobre Australia en esta área que sugiere fuertemente que el comercio de agua, especialmente en los mercados a corto plazo, no solo permite mitigar sustancialmente los impactos de las sequías en la agricultura, sino que ahora los agricultores lo consideran como un medio útil y básico. instrumento en su caja de herramientas de gestión de riesgos. Los desafíos políticos para Australia en el área de la regulación del agua en la agricultura están más en las áreas de cómo ajustar el sistema existente y los problemas de solidez relacionados con el cambio climático. Si bien el ejemplo australiano ilustra las posibles ganancias de eficiencia que surgen del comercio de agua, especialmente para gestionar la escasez de agua en circunstancias de sequía, la elección de un instrumento siempre debe considerar la gama completa de costos y beneficios.",6
2,"Los procesos de descentralización, emprendidos para una serie de actividades que antes realizaba el gobierno central, llevaron a modificaciones importantes en la forma en que se administraba y operaba el sector del agua. Las únicas excepciones a esta tendencia fueron Azerbaiyán, Bielorrusia y Turkmenistán, donde la gestión del sector del agua ha permanecido fuertemente centralizada. En Azerbaiyán, por ejemplo, la empresa nacional AZERSU JSC suministra agua al 65 % de la población, con un 10 % proporcionado por pequeños operadores privados y un 25 % por los propios hogares.",6
3,"Esto puede tener consecuencias sustanciales para la calidad del agua, especialmente en el río Mississippi y el norte del Golfo de México (De la Torre Ugarte et al., Si se requieren aportes de nutrientes para las plantaciones de madera, la infiltración y la escorrentía de nitrógeno también pueden representar un riesgo para las aguas subterráneas). (Lattimore et al., otra conclusión importante es que la ubicación de la producción y el tipo de producción de labranza, el sistema de rotación de cultivos y otras prácticas de gestión agrícola utilizadas en la producción de materias primas para la producción de bioenergía también influirán en gran medida en la calidad del agua (De la Torre Ugarte et al. ,",6
4,"La función de beneficio también incorpora parámetros que afectan a todos los usuarios, como los precios de insumos y productos y el clima.7 Por ejemplo, en un año seco, se espera que el beneficio de aplicar cualquier cantidad de agua a un cultivo sea mayor que en un año seco. año húmedo. La optimización óptima requeriría una valoración explícita de todos los posibles servicios ambientales asociados con el recurso de agua subterránea, tanto ahora como en el futuro. En cambio, la sociedad decide el nivel de servicios hidrológicos que es deseable y el problema económico es lograr ese nivel utilizando una solución que maximice los beneficios para los usuarios del agua.",6


Revisamos que se hayan corregido los carácteres especiales para que no aparezcan problemas de tildes ni acentos circunflejos extraños.

In [14]:
data.iloc[5]

Textos_espanol    La descentralización también implica dar mayor protagonismo a los Comités Técnicos de Aguas Subterráneas (COTAS) en la gestión de los acuÃferos, fortalecer la organización y funcionamiento de los Consejos de Cuenca y sus órganos auxiliares, y consolidar las funciones de gobernanza y organización regional de la CONAGUA. No existe un modelo único para mejorar la gobernanza pública en el sector hÃdrico y superar los obstáculos institucionales para su implementación, pero es fundamental diagnosticar los principales retos de gobernanza que se experimentan entre los ministerios y organismos públicos, entre los niveles de gobierno y entre los actores subnacionales, y revisar los instrumentos de gobernanza existentes. Esto ayudará a definir claramente las funciones y responsabilidades de las autoridades públicas y a establecer mecanismos de resolución de conflictos.
sdg                                                                                                            

## Preparación de datos para entrenamiento
Para esta parte vamos a estandarizar los datos para que cumplan con un cierto valor estandar para que el modelo pueda entrenar de manera correcta. Para esto vamos a colocar todos los textos en minúsculas y vamos a eliminar todos los signos de puntuación, en adición a eso vamos a eliminar los stopwords. Para terminar transformaremos los números en dígitos a números en letras.

In [15]:
# Transformación a minuscúlas
data['Textos_espanol'] = data['Textos_espanol'].str.lower()

In [16]:
# Remover signos de puntuaación
def remove_punctuation(text):
    return re.sub(r'[^\w\s]', ' ', text)


data['Textos_espanol'] = data['Textos_espanol'].apply(remove_punctuation)

In [17]:
# Transformación de números a texto

# En español
def replace_numbers_with_spanish_text(text):
    words = text.split()
    for i, word in enumerate(words):
        if word.isdigit():
            words[i] = num2words(int(word), lang='es')
    return ' '.join(words)

# En inglés
# p = inflect.engine()
# def replace_numbers_with_text(text):
#     words = text.split()
#     for i, word in enumerate(words):
#         if word.isdigit():
#             words[i] = p.number_to_words(word)
#     return ' '.join(words)

data['Textos_espanol'] = data['Textos_espanol'].apply(replace_numbers_with_spanish_text)

In [18]:
# Eliminar los stop_words de la columna Textos_espanol
def remove_stop_words(text):
    words = word_tokenize(text)
    return ' '.join([word for word in words if word not in stop_words])

data['Textos_espanol'] = data['Textos_espanol'].apply(remove_stop_words)

In [19]:
#Revisión de los datos luego de la limpieza
data.head(5)

,Textos_espanol,sdg
0,importante destacar año sequía espera disminuyan aportes aumente demanda agua cultivos vegetación provocará reducción caudal ríos conectados hidrológicamente acuífero así menor disponibilidad agua ecosistemas dependientes aguas subterráneas externalidades espaciales pueden incluir agotamiento cursos agua descenso nivel freático local ejemplo bombeo continuo pozo b provocará agotamiento cursos agua reducción caudal,6
1,gran cantidad literatura australia área sugiere fuertemente comercio agua especialmente mercados corto plazo solo permite mitigar sustancialmente impactos sequías agricultura sino ahora agricultores consideran medio útil básico instrumento caja herramientas gestión riesgos desafíos políticos australia área regulación agua agricultura áreas cómo ajustar sistema existente problemas solidez relacionados cambio climático si bien ejemplo australiano ilustra posibles ganancias eficiencia surgen comercio agua especialmente gestionar escasez agua circunstancias sequía elección instrumento siempre debe considerar gama completa costos beneficios,6
2,procesos descentralización emprendidos serie actividades realizaba gobierno central llevaron modificaciones importantes forma administraba operaba sector agua únicas excepciones tendencia azerbaiyán bielorrusia turkmenistán gestión sector agua permanecido fuertemente centralizada azerbaiyán ejemplo empresa nacional azersu jsc suministra agua sesenta cinco población diez proporcionado pequeños operadores privados veinticinco propios hogares,6
3,puede tener consecuencias sustanciales calidad agua especialmente río mississippi norte golfo méxico torre ugarte et si requieren aportes nutrientes plantaciones madera infiltración escorrentía nitrógeno pueden representar riesgo aguas subterráneas lattimore et conclusión importante ubicación producción tipo producción labranza sistema rotación cultivos prácticas gestión agrícola utilizadas producción materias primas producción bioenergía influirán gran medida calidad agua torre ugarte et,6
4,función beneficio incorpora parámetros afectan usuarios precios insumos productos clima siete ejemplo año seco espera beneficio aplicar cualquier cantidad agua cultivo mayor año seco año húmedo optimización óptima requeriría valoración explícita posibles servicios ambientales asociados recurso agua subterránea ahora futuro cambio sociedad decide nivel servicios hidrológicos deseable problema económico lograr nivel utilizando solución maximice beneficios usuarios agua,6


In [20]:
"df.carvajal@uniandes.edu.co"

'df.carvajal@uniandes.edu.co'